## E-commerce — Выявление профилей потребления в магазине «Пока все ещё тут».

### Цель - выработка таргетированных маркетинговых стратегий отдельно для каждого сегмента покупателей  
- формирование для каждого сегмента специальных предолжений  
    * товаров, которые интересны пользователям  
    * дополняющих товаров, которые могли бы пользователя заинтересовать
- предложение специальных условий обслуживания  и прочее

In [99]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import time
import phik

import matplotlib as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px 
import plotly.graph_objects as go
from os import path

import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('russian')
from sklearn.feature_extraction.text import TfidfVectorizer
# import spacy
# from spacy.lang.ru.examples import sentences 
import ru_core_news_sm
nlp = ru_core_news_sm.load()
# nlp = spacy.load("ru_core_news_sm")
from scipy import stats as st
import math as mth
from xgboost import  XGBClassifier, plot_importance
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn import set_config
from sklearn.utils import shuffle
from numpy.random import RandomState
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, make_scorer, accuracy_score, f1_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline as sk_Pipeline
from imblearn.pipeline import Pipeline 
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer

from catboost import Pool, CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.3f}'.format)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kbzunder/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
path_local = '/Users/kbzunder/Downloads/'
path_train = '/datasets/'
if path.exists(path_local):
    data = pd.read_csv(path_local+'ecommerce_dataset.csv')
else:
    data = pd.read_csv(path_train+'ecommerce_datasets.csv')


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6737 entries, 0 to 6736
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         6737 non-null   int64  
 1   customer_id  6737 non-null   object 
 2   order_id     6737 non-null   int64  
 3   product      6737 non-null   object 
 4   quantity     6737 non-null   int64  
 5   price        6737 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 315.9+ KB


В нашем распоряжении ограниченный набор данных, не включающий в себя информцию о возрасте, поле, географическом положении покупателя, поэтому  возможности сегментации ограничены предположительно следюущими типами:   
- покупатель, совершивший одну покупку / вернувшийся покупатель  
- количество покупок
- среднее время между покупками  
- LTV / средний чек  
- время совершения покупки (сезон/будни/выходные/время дня)  
- тип покупки  

Поскольку столбец product не содержит точного указания на тип товара, предполагаю использовать NLP модель для формирования признаков и последующего проведения кластеризации для выявления групп товаров.

### План работы над проектом  
1. Знакомство с данными и предобработка  
    - изменение типов данных, выявление дубликатов
2. Исследование данных
    - аномалии, закономерности
3. Разделение пользователей на сегменты, анализ ключевых метрик внутри сегмента: LTV, средний чек, время между покупками
4. Формулировка и проверка  гипотез:
    - H0: LTV/средний чек/время между покупками  одинаковы между сегментами
    - H1: LTV/средний чек/время между покупками различны между сегментами
5. Анализ результатов исследования  
6. Формулирование выводов и предложений  
7. Формирования дашборда и презентации для визуализации выводов и предложений

In [5]:
data.head()

,date,customer_id,order_id,product,quantity,price
0,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Алое Вера, d12, h30",1,142.000
1,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Кофе Арабика, d12,...",1,194.000
2,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Радермахера d-12 см h-20 см,1,112.000
3,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Хризолидокарпус Лутесценс d-9 см,1,179.000
4,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Циперус Зумула d-12 см h-25 см,1,112.000


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6737 entries, 0 to 6736
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         6737 non-null   int64  
 1   customer_id  6737 non-null   object 
 2   order_id     6737 non-null   int64  
 3   product      6737 non-null   object 
 4   quantity     6737 non-null   int64  
 5   price        6737 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 315.9+ KB


In [7]:
#Поменяем формат даты:
data['date'] = pd.to_datetime(data['date'], format='%Y%m%d%H')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6737 entries, 0 to 6736
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         6737 non-null   datetime64[ns]
 1   customer_id  6737 non-null   object        
 2   order_id     6737 non-null   int64         
 3   product      6737 non-null   object        
 4   quantity     6737 non-null   int64         
 5   price        6737 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 315.9+ KB


In [9]:
data.describe()

,order_id,quantity,price
count,6737.000,6737.000,6737.000
mean,43128.948,2.502,462.028
std,27899.415,15.266,871.296
min,12624.000,1.000,9.000
25%,14827.000,1.000,101.000
50%,68503.000,1.000,135.000
75%,70504.000,1.000,398.000
max,73164.000,1000.000,14917.000


In [10]:
data[data['quantity']==1000]

,date,customer_id,order_id,product,quantity,price
5456,2019-06-18 15:00:00,312e9a3e-5fca-43ff-a6a1-892d2b2d5ba6,71743,"Вантуз с деревянной ручкой d14 см красный, Bur...",1000,675.000


После обращения к оставщику данных, выяснено, что это тестовый зкаказ, его следует убрать из выборки

In [11]:
data = data.query('order_id != 71743')

In [12]:
data.describe()

,order_id,quantity,price
count,6736.000,6736.000,6736.000
mean,43124.700,2.353,461.997
std,27899.307,9.238,871.357
min,12624.000,1.000,9.000
25%,14827.000,1.000,101.000
50%,68503.000,1.000,135.000
75%,70503.250,1.000,397.250
max,73164.000,334.000,14917.000


In [13]:
data.duplicated().sum()

0

В данных отсутсвуют явные дубликаты

In [14]:
data['product'] = data['product'].apply(lambda x: x.lower())

In [15]:
data['revenue'] = data['price'] * data['quantity']

In [16]:
(data['date'].min(), data['date'].max())

(Timestamp('2018-10-01 00:00:00'), Timestamp('2019-10-31 16:00:00'))

In [46]:
#добавим признак месяца и года
data['year_month'] = data['date'].dt.to_period('M')

In [48]:
# а также месяца отдельно
data['month'] = data['date'].dt.month

In [49]:
data.head()

,date,customer_id,order_id,product,quantity,price,revenue,year_month,month
0,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"комнатное растение в горшке алое вера, d12, h30",1,142.000,142.000,2018-10,10
1,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"комнатное растение в горшке кофе арабика, d12,...",1,194.000,194.000,2018-10,10
2,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,радермахера d-12 см h-20 см,1,112.000,112.000,2018-10,10
3,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,хризолидокарпус лутесценс d-9 см,1,179.000,179.000,2018-10,10
4,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,циперус зумула d-12 см h-25 см,1,112.000,112.000,2018-10,10


In [53]:
# посмотрим, как меняется количество покупок по месяцам:
data_by_months = data.groupby('year_month', as_index=False).agg({'quantity': 'sum'}).sort_values(by='year_month')


In [68]:
data_by_months['year_month'] = data_by_months['year_month'].astype(str)

In [63]:
revenue_by_month = data.groupby('year_month', as_index=False).agg({'revenue': 'sum'}).sort_values(by='year_month')

In [66]:
revenue_by_month['year_month'] = revenue_by_month['year_month'].astype(str)

In [71]:
import plotly.graph_objects as go

x = data_by_months['year_month']
y = data_by_months['quantity']

# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
            textposition='auto',
        )]
)
fig.update_layout(
    title="Продажи в штуках по месяцам",
    xaxis_title="месяцы",
    yaxis_title="кол-во товара",
)

fig.show()

In [70]:
import plotly.graph_objects as go

x = revenue_by_month['year_month']
y = revenue_by_month['revenue']

# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
            textposition='auto',
        )])
fig.update_layout(
    title="Продажи в руб по месяцам",
    xaxis_title="месяцы",
    yaxis_title="выручка",
)

fig.show()

In [72]:
orders_by_month = data.groupby('year_month', as_index=False).agg({'order_id': 'count'}).sort_values(by='year_month')

In [73]:
orders_by_month['year_month'] = orders_by_month['year_month'].astype(str)

In [74]:
import plotly.graph_objects as go

x = orders_by_month['year_month']
y = orders_by_month['order_id']

# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
            textposition='auto',
        )])
fig.update_layout(
    title="Количество заказов по месяцам",
    xaxis_title="месяцы",
    yaxis_title="кол-во заказов",
)

fig.show()

В нашем распоряжении всего один год наблюдений, очень сложно увидеть сезонность на таком ограниченном промежутке времени, можно заключить, что в марте-июне значительно растет количество заказов, проданных товаров и выручки, это дает возмжоность выделить покупателей, делающих покупки в этот период (возможно, это товары для огорода, семена, цветы)

In [19]:
df_prod_in_order = data[['order_id','product']].groupby('order_id', as_index=False).agg('count')\
    .rename(columns=({'product': 'prods_in_order'}))\
    .sort_values(by = 'prods_in_order', ascending=False)\
    .reset_index(drop=True)

In [20]:
df_prod_in_order

,order_id,prods_in_order
0,14833,888
1,14835,203
2,14753,90
3,14897,63
4,70960,60
...,...,...
2778,70126,1
2779,70125,1
2780,70123,1
2781,70121,1


In [21]:
fig = px.box(df_prod_in_order, x='prods_in_order')
fig.show()

In [ ]:
В основном заказывют 1 товар, но, вероятно есть оптовые закупки юридических лиц, как, например вешалки, плечики и тп - можно выделить в отдельных сегмент оптовых покупателей

In [22]:
data[data['order_id']==14698].sort_values(by='product')

,date,customer_id,order_id,product,quantity,price,revenue
3195,2019-04-27 16:00:00,d3b9ffea-d143-4747-8d59-74ab843d1ad6,14698,земляника садовая хоней d-9 см p9,1,75.000,75.000
3196,2019-04-27 16:00:00,d3b9ffea-d143-4747-8d59-74ab843d1ad6,14698,"колокольчик персиколистный белый объем 0,5 л",1,105.000,105.000
3197,2019-04-27 16:00:00,d3b9ffea-d143-4747-8d59-74ab843d1ad6,14698,пиретрум робинсон красный объем 1 л,1,112.000,112.000
3198,2019-04-27 16:00:00,d3b9ffea-d143-4747-8d59-74ab843d1ad6,14698,рассада зелени для кухни лаванда блю райдер ди...,1,120.000,120.000
3199,2019-04-27 16:00:00,d3b9ffea-d143-4747-8d59-74ab843d1ad6,14698,рассада зелени для кухни лаванда прованc диам....,1,120.000,120.000
3200,2019-04-27 16:00:00,d3b9ffea-d143-4747-8d59-74ab843d1ad6,14698,рассада зелени для кухни лаванды в горшке диам...,1,120.000,120.000
3201,2019-04-27 16:00:00,d3b9ffea-d143-4747-8d59-74ab843d1ad6,14698,рассада зелени для кухни розмарина в горшке ди...,1,120.000,120.000
3202,2019-04-27 16:00:00,d3b9ffea-d143-4747-8d59-74ab843d1ad6,14698,рассада клубники зенга зенгана в кассете по e6,1,285.000,285.000
3203,2019-04-27 16:00:00,d3b9ffea-d143-4747-8d59-74ab843d1ad6,14698,рассада клубники зенга зенгана горшок 9х9 см p9,1,75.000,75.000
3204,2019-04-27 16:00:00,d3b9ffea-d143-4747-8d59-74ab843d1ad6,14698,"томата (помидор) ""золотая канарейка"" №32 сорт ...",1,38.000,38.000


In [23]:
df_revenu_in_order = data[['order_id','revenue']].groupby('order_id', as_index=False).agg('sum')\
    .rename(columns=({'revenue': 'revenue_per_order'}))\
    .sort_values(by = 'revenue_per_order', ascending=False)\
    .reset_index(drop=True)

In [24]:
df_revenu_in_order

,order_id,revenue_per_order
0,14833,114750.000
1,70960,65220.000
2,68760,50770.000
3,69531,49668.000
4,71668,49432.000
...,...,...
2778,70988,22.000
2779,71178,22.000
2780,68985,15.000
2781,71661,15.000


In [25]:
fig = px.box(df_revenu_in_order, x='revenue_per_order', title = 'Распределение стоимости заказов', labels={'Выручка с одного заказа'})
fig.show()

С одной стороны заказы с выручкой более 4 тыс рублей и количеством позиций в заказе, доходящим до 888 - явные выбросы, с другой стороны, нужно оценить, какую долю всей выручки они составляют, возможно имеет смысл отнести их в премиум сегмент

In [40]:
rev_less_then_4 = df_revenu_in_order.query('revenue_per_order <=4000')['revenue_per_order'].sum()
rev_more_then_4 = df_revenu_in_order.query('revenue_per_order > 4000')['revenue_per_order'].sum()

In [41]:
((rev_more_then_4 / rev_less_then_4)*100.0).round(2)

68.68

In [43]:
orders_cnt_less_then_4 = df_revenu_in_order.query('revenue_per_order <=4000')['order_id'].count()
orders_cnt_more_then_4 = df_revenu_in_order.query('revenue_per_order > 4000')['order_id'].count()

In [45]:
( (orders_cnt_more_then_4 / orders_cnt_less_then_4)*100.0).round(2)

6.96

Видим, что заказы с выручкой более 4000 рублей составляют почти 70 процентов  выручки за весь рассматриваемый период, а доля таких заказов лишь 7%, считаю, таких клиентов необходимо отнести в отдельных сегмент, не считать их данные выбросами

In [27]:
#Посмотрим распределение  количества товаров в  заказе
fig = px.box(data, x='quantity')
fig.show()

In [78]:
# Посмотрим, как наши покупатели деляются на новых и повторных:
orders_per_user = data.groupby('customer_id', as_index=False).agg({'order_id':'nunique'}).rename(columns={'order_id':'orders_per_user'}).\
  sort_values(by='orders_per_user', ascending=False).reset_index(drop=True)

In [81]:
orders_per_user.loc[orders_per_user['orders_per_user']>=2, 'returning'] = 'yes'
orders_per_user.loc[orders_per_user['orders_per_user']<2, 'returning'] = 'no'

In [82]:
orders_per_user

,customer_id,orders_per_user,returning
0,c971fb21-d54c-4134-938f-16b62ee86d3b,126,yes
1,4d93d3f6-8b24-403b-a74b-f5173e40d7db,35,yes
2,73d1cd35-5e5f-4629-8cf2-3fda829d4e58,17,yes
3,b7b865ab-0735-407f-8d0c-31f74d2806cc,7,yes
4,0184f535-b60a-4914-a982-231e3f615206,5,yes
...,...,...,...
2445,58a23390-1590-424c-83eb-b75381eec614,1,no
2446,58a966e2-b773-4ddd-aeff-472f8320a6a3,1,no
2447,58e420e1-e083-4e77-929a-af4d8d0f4c8e,1,no
2448,5901a4c4-768d-4dc5-9d81-3546e29820fb,1,no


In [84]:
orders_per_user.groupby('returning', as_index=False).agg({'customer_id':'nunique'})

,returning,customer_id
0,no,2290
1,yes,160


In [86]:
fig = px.bar(orders_per_user.groupby('returning', as_index=False).agg({'customer_id':'nunique'}).\
    rename(columns={'customer_id':'customers'}), x='returning', y='customers',
    title = 'Распределение покупателей на новых и вернувшихся')
fig.show()

In [88]:
orders_per_user.query('returning =="yes"')['customer_id'].nunique() / orders_per_user['customer_id'].nunique()

0.0653061224489796

Четко видно распределение покупаталей на новых и вернувшихся, вернувшиеся составляют меньше `7%` 

### Исследуем, какие группы товаров покупают в нашем магазине

In [89]:
data.head()

,date,customer_id,order_id,product,quantity,price,revenue,year_month,month
0,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"комнатное растение в горшке алое вера, d12, h30",1,142.000,142.000,2018-10,10
1,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"комнатное растение в горшке кофе арабика, d12,...",1,194.000,194.000,2018-10,10
2,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,радермахера d-12 см h-20 см,1,112.000,112.000,2018-10,10
3,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,хризолидокарпус лутесценс d-9 см,1,179.000,179.000,2018-10,10
4,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,циперус зумула d-12 см h-25 см,1,112.000,112.000,2018-10,10


In [105]:
#Уберем из текста знаки пунктуации, стоп-слова, пробелы

def  preprocess_text(text, stopwords):
    # cleaning punctuation
    text = text.translate(str.maketrans('', string.digits, string.punctuation))

    # text to lower
    text = text.lower()

    # removing stopwords
    text = ' '.join([word for word in text.split() if word not in stopwords])

    # removing whitespaces
    text = re.sub(r'\s', ' ', text).strip()

    return text

In [106]:
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

In [107]:
data['clean'] = data['product'].apply(lambda x: preprocess_text(x, stopwords=stop_words))

TypeError: maketrans expected at most 3 arguments, got 4

In [103]:
data['lemmatized'] = data['clean'].apply(lambda x: lemmatize_text(x) )

In [104]:
data.head()

,date,customer_id,order_id,product,quantity,price,revenue,year_month,month,clean,lemmatized
0,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"комнатное растение в горшке алое вера, d12, h30",1,142.000,142.000,2018-10,10,комнатное растение горшке алое вера d12 h30,комнатный растение горшке алый вера d12 h30
1,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"комнатное растение в горшке кофе арабика, d12,...",1,194.000,194.000,2018-10,10,комнатное растение горшке кофе арабика d12 h25,комнатный растение горшке кофе арабика d12 h25
2,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,радермахера d-12 см h-20 см,1,112.000,112.000,2018-10,10,радермахера d12 см h20 см,радермахера d12 см h20 см
3,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,хризолидокарпус лутесценс d-9 см,1,179.000,179.000,2018-10,10,хризолидокарпус лутесценс d9 см,хризолидокарпус лутесценс d9 см
4,2018-10-01,ee47d746-6d2f-4d3c-9622-c31412542920,68477,циперус зумула d-12 см h-25 см,1,112.000,112.000,2018-10,10,циперус зумула d12 см h25 см,циперус зумула d12 см h25 см
